# Tier 6: One-Class SVM Anomaly Detection

---

**Author:** Brandon Deloatch
**Affiliation:** Quipu Research Labs, LLC
**Date:** 2025-10-02
**Version:** v1.3
**License:** MIT
**Notebook ID:** 82de510c-aad9-4113-92dd-4f88b7ccac51

---

## Citation
Brandon Deloatch, "Tier 6: One-Class SVM Anomaly Detection," Quipu Research Labs, LLC, v1.3, 2025-10-02.

Please cite this notebook if used or adapted in publications, presentations, or derivative work.

---

## Contributors / Acknowledgments
- **Primary Author:** Brandon Deloatch (Quipu Research Labs, LLC)
- **Institutional Support:** Quipu Research Labs, LLC - Advanced Analytics Division
- **Technical Framework:** Built on scikit-learn, pandas, numpy, and plotly ecosystems
- **Methodological Foundation:** Statistical learning principles and modern data science best practices

---

## Version History
| Version | Date | Notes |
|---------|------|-------|
| v1.3 | 2025-10-02 | Enhanced professional formatting, comprehensive documentation, interactive visualizations |
| v1.2 | 2024-09-15 | Updated analysis methods, improved data generation algorithms |
| v1.0 | 2024-06-10 | Initial release with core analytical framework |

---

## Environment Dependencies
- **Python:** 3.8+
- **Core Libraries:** pandas 2.0+, numpy 1.24+, scikit-learn 1.3+
- **Visualization:** plotly 5.0+, matplotlib 3.7+
- **Statistical:** scipy 1.10+, statsmodels 0.14+
- **Development:** jupyter-lab 4.0+, ipywidgets 8.0+

> **Reproducibility Note:** Use requirements.txt or environment.yml for exact dependency matching.

---

## Data Provenance
| Dataset | Source | License | Notes |
|---------|--------|---------|-------|
| Synthetic Data | Generated in-notebook | MIT | Custom algorithms for realistic simulation |
| Statistical Distributions | NumPy/SciPy | BSD-3-Clause | Standard library implementations |
| ML Algorithms | Scikit-learn | BSD-3-Clause | Industry-standard implementations |
| Visualization Schemas | Plotly | MIT | Interactive dashboard frameworks |

---

## Execution Provenance Logs
- **Created:** 2025-10-02
- **Notebook ID:** 82de510c-aad9-4113-92dd-4f88b7ccac51
- **Execution Environment:** Jupyter Lab / VS Code
- **Computational Requirements:** Standard laptop/workstation (2GB+ RAM recommended)

> **Auto-tracking:** Execution metadata can be programmatically captured for reproducibility.

---

## Disclaimer & Responsible Use
This notebook is provided "as-is" for educational, research, and professional development purposes. Users assume full responsibility for any results, applications, or decisions derived from this analysis.

**Professional Standards:**
- Validate all results against domain expertise and additional data sources
- Respect licensing and attribution requirements for all dependencies
- Follow ethical guidelines for data analysis and algorithmic decision-making
- Credit all methodological sources and derivative frameworks appropriately

**Academic & Commercial Use:**
- Permitted under MIT license with proper attribution
- Suitable for educational curriculum and professional training
- Appropriate for commercial adaptation with citation requirements
- Recommended for reproducible research and transparent analytics

---



In [ ]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from sklearn.svm import OneClassSVM
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import classification_report, precision_score, recall_score, f1_score
from sklearn.datasets import make_blobs, make_moons
from sklearn.decomposition import PCA
import warnings
warnings.filterwarnings('ignore')

print(" Tier 6: One-Class SVM Anomaly Detection - Libraries Loaded!")
print("=" * 60)
print("One-Class SVM Techniques:")
print("• Boundary-based anomaly detection with support vectors")
print("• Kernel trick for non-linear decision boundaries")
print("• Nu parameter tuning for outlier fraction control")
print("• High-dimensional anomaly detection capabilities")
print("• Novelty detection for previously unseen data patterns")

In [ ]:
# Generate comprehensive One-Class SVM datasets
np.random.seed(42)

# 1. Network intrusion detection dataset
n_normal_connections = 4000
n_intrusions = 200

# Normal network traffic patterns
normal_connections = []
for i in range(n_normal_connections):
 # Simulate normal network behavior
 packet_size = np.random.normal(1500, 300) # Standard MTU size
 connection_duration = np.random.exponential(30) # Typical web session
 bytes_sent = np.random.lognormal(8, 1.5) # Normal data transfer
 bytes_received = bytes_sent * np.random.normal(1.2, 0.3) # Response ratio
 port_number = np.random.choice([80, 443, 22, 25, 53], p=[0.4, 0.3, 0.1, 0.1, 0.1])

 # Protocol behavior features
 tcp_flags = np.random.randint(0, 8) # Normal TCP flag combinations
 packet_count = connection_duration * np.random.poisson(5) # Packets per second

 normal_connections.append({
 'connection_id': f'NORM_{i:06d}',
 'packet_size': packet_size,
 'duration': connection_duration,
 'bytes_sent': bytes_sent,
 'bytes_received': bytes_received,
 'port': port_number,
 'tcp_flags': tcp_flags,
 'packet_count': packet_count,
 'protocol_anomaly_score': np.random.beta(2, 8), # Low anomaly score
 'is_intrusion': 0
 })

# Intrusion patterns (anomalies)
intrusion_connections = []
intrusion_types = ['DDoS', 'Port_Scan', 'Data_Exfiltration', 'Malware_C2']

for i in range(n_intrusions):
 intrusion_type = np.random.choice(intrusion_types)

 if intrusion_type == 'DDoS':
 # High packet count, short duration, small packets
 packet_size = np.random.normal(64, 20) # Small packets
 connection_duration = np.random.exponential(2) # Very short
 bytes_sent = np.random.normal(500, 100)
 bytes_received = bytes_sent * 0.1 # Minimal response
 packet_count = np.random.poisson(1000) # Very high packet rate

 elif intrusion_type == 'Port_Scan':
 # Many different ports, minimal data
 packet_size = np.random.normal(100, 30)
 connection_duration = np.random.exponential(0.5) # Very quick
 bytes_sent = np.random.normal(50, 20)
 bytes_received = 0 # No response typically
 packet_count = np.random.poisson(2)

 elif intrusion_type == 'Data_Exfiltration':
 # Large data transfers, unusual patterns
 packet_size = np.random.normal(1500, 100)
 connection_duration = np.random.exponential(300) # Long sessions
 bytes_sent = np.random.lognormal(12, 2) # Very large transfers
 bytes_received = bytes_sent * 0.1 # Minimal response
 packet_count = connection_duration * np.random.poisson(20)

 else: # Malware_C2
 # Regular but suspicious patterns
 packet_size = np.random.normal(200, 50)
 connection_duration = np.random.exponential(60)
 bytes_sent = np.random.normal(1000, 300)
 bytes_received = bytes_sent * np.random.normal(0.5, 0.2)
 packet_count = np.random.poisson(30)

 port_number = np.random.randint(1024, 65535) if intrusion_type == 'Port_Scan' else np.random.choice([80, 443])
 tcp_flags = np.random.randint(8, 16) # Unusual flag combinations

 intrusion_connections.append({
 'connection_id': f'{intrusion_type}_{i:06d}',
 'packet_size': packet_size,
 'duration': connection_duration,
 'bytes_sent': bytes_sent,
 'bytes_received': bytes_received,
 'port': port_number,
 'tcp_flags': tcp_flags,
 'packet_count': packet_count,
 'protocol_anomaly_score': np.random.beta(8, 2), # High anomaly score
 'is_intrusion': 1,
 'intrusion_type': intrusion_type
 })

# Combine datasets
all_connections = normal_connections + intrusion_connections
network_df = pd.DataFrame(all_connections).sample(frac=1).reset_index(drop=True)

print(" Network Intrusion Dataset Created:")
print(f"Total connections: {len(network_df)}")
print(f"Intrusion connections: {n_intrusions} ({n_intrusions/len(network_df)*100:.1f}%)")
print(f"Intrusion types: {network_df[network_df['is_intrusion']==1]['intrusion_type'].value_counts().to_dict()}")

# 2. Industrial equipment sensor dataset
n_normal_readings = 3500
n_fault_readings = 175

# Normal equipment operation
normal_readings = []
for i in range(n_normal_readings):
 # Normal operating parameters with correlations
 temperature = np.random.normal(75, 5) # °C
 pressure = np.random.normal(100, 8) # PSI
 vibration = np.random.normal(2, 0.5) # mm/s
 flow_rate = np.random.normal(50, 5) # L/min

 # Correlated parameters
 power_consumption = 100 + 0.8 * temperature + 0.3 * pressure + np.random.normal(0, 3)
 efficiency = 95 - 0.1 * temperature + 0.05 * flow_rate + np.random.normal(0, 2)

 normal_readings.append({
 'reading_id': f'NORMAL_{i:06d}',
 'temperature': temperature,
 'pressure': pressure,
 'vibration': vibration,
 'flow_rate': flow_rate,
 'power_consumption': power_consumption,
 'efficiency': efficiency,
 'is_fault': 0
 })

# Equipment fault patterns
fault_readings = []
fault_types = ['Overheating', 'Pressure_Leak', 'Bearing_Wear', 'Flow_Blockage']

for i in range(n_fault_readings):
 fault_type = np.random.choice(fault_types)

 if fault_type == 'Overheating':
 temperature = np.random.normal(95, 8) # High temperature
 pressure = np.random.normal(100, 8)
 vibration = np.random.normal(2.5, 0.8) # Slight increase
 flow_rate = np.random.normal(48, 5) # Slight decrease

 elif fault_type == 'Pressure_Leak':
 temperature = np.random.normal(75, 5)
 pressure = np.random.normal(80, 10) # Low pressure
 vibration = np.random.normal(3, 1) # Higher vibration
 flow_rate = np.random.normal(45, 8) # Reduced flow

 elif fault_type == 'Bearing_Wear':
 temperature = np.random.normal(80, 6) # Elevated temperature
 pressure = np.random.normal(100, 8)
 vibration = np.random.normal(8, 2) # Very high vibration
 flow_rate = np.random.normal(50, 5)

 else: # Flow_Blockage
 temperature = np.random.normal(78, 5)
 pressure = np.random.normal(120, 15) # High pressure
 vibration = np.random.normal(2, 0.5)
 flow_rate = np.random.normal(30, 10) # Very low flow

 power_consumption = 100 + 0.8 * temperature + 0.3 * pressure + np.random.normal(10, 5) # Higher variance
 efficiency = 85 - 0.2 * temperature + 0.02 * flow_rate + np.random.normal(0, 5) # Lower efficiency

 fault_readings.append({
 'reading_id': f'{fault_type}_{i:06d}',
 'temperature': temperature,
 'pressure': pressure,
 'vibration': vibration,
 'flow_rate': flow_rate,
 'power_consumption': power_consumption,
 'efficiency': efficiency,
 'is_fault': 1,
 'fault_type': fault_type
 })

# Combine equipment data
all_readings = normal_readings + fault_readings
equipment_df = pd.DataFrame(all_readings).sample(frac=1).reset_index(drop=True)

print(f"\nIndustrial Equipment Dataset Created:")
print(f"Total readings: {len(equipment_df)}")
print(f"Fault readings: {n_fault_readings} ({n_fault_readings/len(equipment_df)*100:.1f}%)")
print(f"Fault types: {equipment_df[equipment_df['is_fault']==1]['fault_type'].value_counts().to_dict()}")

# 3. Create 2D synthetic dataset for visualization
X_synthetic, _ = make_blobs(n_samples=800, centers=1, cluster_std=1.0, random_state=42)
# Add outliers
n_outliers = 40
outliers = np.random.uniform(low=-6, high=6, size=(n_outliers, 2))
X_synthetic_with_outliers = np.vstack([X_synthetic, outliers])
y_synthetic = np.hstack([np.ones(len(X_synthetic)), -np.ones(n_outliers)])

print(f"\nSynthetic 2D Dataset Created:")
print(f"Normal points: {len(X_synthetic)}")
print(f"Outlier points: {n_outliers}")

In [ ]:
# 1. ONE-CLASS SVM MODEL DEVELOPMENT AND TUNING
print(" 1. ONE-CLASS SVM MODEL DEVELOPMENT AND TUNING")
print("=" * 48)

# Prepare network intrusion data
network_features = ['packet_size', 'duration', 'bytes_sent', 'bytes_received',
 'packet_count', 'protocol_anomaly_score']
X_network = network_df[network_features].values
y_network = network_df['is_intrusion'].values

# Split into normal training data and test data
X_normal = X_network[y_network == 0] # Only normal data for training
X_test = X_network # Full dataset for testing
y_test = y_network

# Standardize features
scaler_network = StandardScaler()
X_normal_scaled = scaler_network.fit_transform(X_normal)
X_test_scaled = scaler_network.transform(X_test)

print(f"Network Data Preparation:")
print(f"Training samples (normal only): {len(X_normal_scaled)}")
print(f"Test samples (normal + intrusions): {len(X_test_scaled)}")

# One-Class SVM parameter tuning
param_grid = {
 'kernel': ['rbf', 'linear', 'poly', 'sigmoid'],
 'gamma': ['scale', 'auto', 0.001, 0.01, 0.1, 1],
 'nu': [0.01, 0.05, 0.1, 0.2] # Expected fraction of outliers
}

print(f"\nParameter Grid Search:")
best_score = -1
best_params = None
best_model = None

# Manual grid search (since One-Class SVM doesn't work well with standard CV)
results = []

for kernel in param_grid['kernel']:
 for gamma in param_grid['gamma']:
 for nu in param_grid['nu']:
 try:
 # Train One-Class SVM
 ocsvm = OneClassSVM(kernel=kernel, gamma=gamma, nu=nu, random_state=42)
 ocsvm.fit(X_normal_scaled)

 # Predict on test set
 predictions = ocsvm.predict(X_test_scaled)
 predictions_binary = (predictions == -1).astype(int) # Convert to binary (1 = anomaly)

 # Calculate performance metrics
 precision = precision_score(y_test, predictions_binary, zero_division=0)
 recall = recall_score(y_test, predictions_binary, zero_division=0)
 f1 = f1_score(y_test, predictions_binary, zero_division=0)

 results.append({
 'kernel': kernel,
 'gamma': gamma,
 'nu': nu,
 'precision': precision,
 'recall': recall,
 'f1': f1
 })

 if f1 > best_score:
 best_score = f1
 best_params = {'kernel': kernel, 'gamma': gamma, 'nu': nu}
 best_model = ocsvm

 print(f"kernel={kernel}, gamma={gamma}, nu={nu}: P={precision:.3f}, R={recall:.3f}, F1={f1:.3f}")

 except Exception as e:
 print(f"Error with kernel={kernel}, gamma={gamma}, nu={nu}: {str(e)}")

print(f"\nBest One-Class SVM parameters: {best_params}")
print(f"Best F1 score: {best_score:.3f}")

# Apply best model to network data
best_predictions = best_model.predict(X_test_scaled)
best_predictions_binary = (best_predictions == -1).astype(int)
decision_scores = best_model.decision_function(X_test_scaled)

network_df['ocsvm_prediction'] = best_predictions_binary
network_df['decision_score'] = decision_scores

# Equipment fault detection
equipment_features = ['temperature', 'pressure', 'vibration', 'flow_rate',
 'power_consumption', 'efficiency']
X_equipment = equipment_df[equipment_features].values
y_equipment = equipment_df['is_fault'].values

X_equipment_normal = X_equipment[y_equipment == 0]
scaler_equipment = StandardScaler()
X_equipment_normal_scaled = scaler_equipment.fit_transform(X_equipment_normal)
X_equipment_scaled = scaler_equipment.transform(X_equipment)

# Train equipment One-Class SVM with best parameters from network analysis
equipment_ocsvm = OneClassSVM(**best_params, random_state=42)
equipment_ocsvm.fit(X_equipment_normal_scaled)

equipment_predictions = equipment_ocsvm.predict(X_equipment_scaled)
equipment_predictions_binary = (equipment_predictions == -1).astype(int)
equipment_decision_scores = equipment_ocsvm.decision_function(X_equipment_scaled)

equipment_df['ocsvm_prediction'] = equipment_predictions_binary
equipment_df['decision_score'] = equipment_decision_scores

# Equipment performance
equipment_precision = precision_score(y_equipment, equipment_predictions_binary, zero_division=0)
equipment_recall = recall_score(y_equipment, equipment_predictions_binary, zero_division=0)
equipment_f1 = f1_score(y_equipment, equipment_predictions_binary, zero_division=0)

print(f"\nEquipment Fault Detection Performance:")
print(f"Precision: {equipment_precision:.3f}")
print(f"Recall: {equipment_recall:.3f}")
print(f"F1-Score: {equipment_f1:.3f}")

# Synthetic 2D data for visualization
scaler_synthetic = StandardScaler()
X_synthetic_scaled = scaler_synthetic.fit_transform(X_synthetic_with_outliers)
X_normal_synthetic = X_synthetic_scaled[y_synthetic == 1]

synthetic_ocsvm = OneClassSVM(kernel='rbf', gamma=0.1, nu=0.05, random_state=42)
synthetic_ocsvm.fit(X_normal_synthetic)

synthetic_predictions = synthetic_ocsvm.predict(X_synthetic_scaled)
synthetic_decision_scores = synthetic_ocsvm.decision_function(X_synthetic_scaled)

print(f"\nSynthetic Data Results:")
print(f"Normal points detected as normal: {np.sum((y_synthetic == 1) & (synthetic_predictions == 1))}")
print(f"Outliers detected as anomalies: {np.sum((y_synthetic == -1) & (synthetic_predictions == -1))}")

In [ ]:
# 2. INTERACTIVE ONE-CLASS SVM VISUALIZATIONS
print(" 2. INTERACTIVE ONE-CLASS SVM VISUALIZATIONS")
print("=" * 44)

# Create comprehensive One-Class SVM dashboard
fig = make_subplots(
 rows=3, cols=2,
 subplot_titles=[
 'One-Class SVM Decision Boundary (2D Synthetic)',
 'Network Intrusion Detection Results',
 'Parameter Tuning Performance Heatmap',
 'Equipment Fault Detection by Sensor Type',
 'Decision Score Distributions',
 'ROC-Style Performance Analysis'
 ],
 specs=[[{"secondary_y": False}, {"secondary_y": False}],
 [{"secondary_y": False}, {"secondary_y": False}],
 [{"secondary_y": False}, {"secondary_y": False}]]
)

# 1. 2D Decision boundary visualization
# Create decision boundary mesh
h = 0.1
x_min, x_max = X_synthetic_scaled[:, 0].min() - 1, X_synthetic_scaled[:, 0].max() + 1
y_min, y_max = X_synthetic_scaled[:, 1].min() - 1, X_synthetic_scaled[:, 1].max() + 1
xx, yy = np.meshgrid(np.arange(x_min, x_max, h), np.arange(y_min, y_max, h))

# Get decision function values for the mesh
mesh_points = np.c_[xx.ravel(), yy.ravel()]
Z = synthetic_ocsvm.decision_function(mesh_points)
Z = Z.reshape(xx.shape)

# Plot decision boundary contour
fig.add_trace(
 go.Contour(
 x=np.arange(x_min, x_max, h),
 y=np.arange(y_min, y_max, h),
 z=Z,
 levels=[0],
 line=dict(color='red', width=3),
 showscale=False,
 name='Decision Boundary'
 ),
 row=1, col=1
)

# Plot normal points
normal_mask = y_synthetic == 1
fig.add_trace(
 go.Scatter(
 x=X_synthetic_scaled[normal_mask, 0],
 y=X_synthetic_scaled[normal_mask, 1],
 mode='markers',
 name='Normal Points',
 marker=dict(color='blue', size=6, opacity=0.7),
 hovertemplate='Normal<br>X: %{x:.2f}<br>Y: %{y:.2f}<extra></extra>'
 ),
 row=1, col=1
)

# Plot outliers
outlier_mask = y_synthetic == -1
fig.add_trace(
 go.Scatter(
 x=X_synthetic_scaled[outlier_mask, 0],
 y=X_synthetic_scaled[outlier_mask, 1],
 mode='markers',
 name='True Outliers',
 marker=dict(color='red', size=8, opacity=0.8, symbol='x'),
 hovertemplate='Outlier<br>X: %{x:.2f}<br>Y: %{y:.2f}<extra></extra>'
 ),
 row=1, col=1
)

# 2. Network intrusion detection results
normal_connections = network_df[network_df['is_intrusion'] == 0]
intrusion_connections = network_df[network_df['is_intrusion'] == 1]

fig.add_trace(
 go.Scatter(
 x=normal_connections['bytes_sent'],
 y=normal_connections['duration'],
 mode='markers',
 name='Normal Connections',
 marker=dict(color='green', size=4, opacity=0.6),
 hovertemplate='Normal<br>Bytes Sent: %{x:,.0f}<br>Duration: %{y:.1f}s<extra></extra>'
 ),
 row=1, col=2
)

fig.add_trace(
 go.Scatter(
 x=intrusion_connections['bytes_sent'],
 y=intrusion_connections['duration'],
 mode='markers',
 name='Intrusions',
 marker=dict(color='red', size=6, opacity=0.8),
 text=intrusion_connections['intrusion_type'],
 hovertemplate='%{text}<br>Bytes Sent: %{x:,.0f}<br>Duration: %{y:.1f}s<extra></extra>'
 ),
 row=1, col=2
)

# 3. Parameter tuning heatmap
results_df = pd.DataFrame(results)
# Create pivot table for heatmap
heatmap_data = results_df.pivot_table(values='f1', index='nu', columns='gamma', aggfunc='max')

fig.add_trace(
 go.Heatmap(
 z=heatmap_data.values,
 x=heatmap_data.columns,
 y=heatmap_data.index,
 colorscale='Viridis',
 showscale=True,
 hovertemplate='Gamma: %{x}<br>Nu: %{y}<br>F1-Score: %{z:.3f}<extra></extra>'
 ),
 row=2, col=1
)

# 4. Equipment fault detection by sensor type
fault_types = equipment_df[equipment_df['is_fault'] == 1]['fault_type'].unique()
colors = ['red', 'orange', 'purple', 'brown']

for i, fault_type in enumerate(fault_types):
 fault_data = equipment_df[equipment_df.get('fault_type') == fault_type]

 fig.add_trace(
 go.Scatter(
 x=fault_data['temperature'],
 y=fault_data['vibration'],
 mode='markers',
 name=f'{fault_type}',
 marker=dict(color=colors[i % len(colors)], size=8, opacity=0.8),
 hovertemplate=f'{fault_type}<br>Temperature: %{{x:.1f}}°C<br>Vibration: %{{y:.1f}} mm/s<extra></extra>'
 ),
 row=2, col=2
 )

# Normal equipment data
normal_equipment = equipment_df[equipment_df['is_fault'] == 0]
fig.add_trace(
 go.Scatter(
 x=normal_equipment['temperature'],
 y=normal_equipment['vibration'],
 mode='markers',
 name='Normal Operation',
 marker=dict(color='green', size=4, opacity=0.5),
 hovertemplate='Normal<br>Temperature: %{x:.1f}°C<br>Vibration: %{y:.1f} mm/s<extra></extra>'
 ),
 row=2, col=2
)

# 5. Decision score distributions
fig.add_trace(
 go.Histogram(
 x=network_df[network_df['is_intrusion'] == 0]['decision_score'],
 name='Normal Scores',
 opacity=0.7,
 nbinsx=30,
 marker_color='blue'
 ),
 row=3, col=1
)

fig.add_trace(
 go.Histogram(
 x=network_df[network_df['is_intrusion'] == 1]['decision_score'],
 name='Intrusion Scores',
 opacity=0.7,
 nbinsx=30,
 marker_color='red'
 ),
 row=3, col=1
)

# 6. Performance analysis across different thresholds
thresholds = np.linspace(-2, 1, 50)
precisions = []
recalls = []
f1_scores = []

for threshold in thresholds:
 threshold_predictions = (network_df['decision_score'] < threshold).astype(int)

 precision = precision_score(network_df['is_intrusion'], threshold_predictions, zero_division=0)
 recall = recall_score(network_df['is_intrusion'], threshold_predictions, zero_division=0)
 f1 = f1_score(network_df['is_intrusion'], threshold_predictions, zero_division=0)

 precisions.append(precision)
 recalls.append(recall)
 f1_scores.append(f1)

fig.add_trace(
 go.Scatter(
 x=thresholds,
 y=precisions,
 mode='lines',
 name='Precision',
 line=dict(color='blue', width=2)
 ),
 row=3, col=2
)

fig.add_trace(
 go.Scatter(
 x=thresholds,
 y=recalls,
 mode='lines',
 name='Recall',
 line=dict(color='red', width=2)
 ),
 row=3, col=2
)

fig.add_trace(
 go.Scatter(
 x=thresholds,
 y=f1_scores,
 mode='lines',
 name='F1-Score',
 line=dict(color='green', width=3)
 ),
 row=3, col=2
)

# Update layout
fig.update_layout(
 height=1200,
 title="One-Class SVM Anomaly Detection Dashboard",
 showlegend=True
)

# Update axis labels
fig.update_xaxes(title_text="Feature 1", row=1, col=1)
fig.update_xaxes(title_text="Bytes Sent", row=1, col=2)
fig.update_xaxes(title_text="Gamma Parameter", row=2, col=1)
fig.update_xaxes(title_text="Temperature (°C)", row=2, col=2)
fig.update_xaxes(title_text="Decision Score", row=3, col=1)
fig.update_xaxes(title_text="Decision Threshold", row=3, col=2)

fig.update_yaxes(title_text="Feature 2", row=1, col=1)
fig.update_yaxes(title_text="Connection Duration (s)", row=1, col=2)
fig.update_yaxes(title_text="Nu Parameter", row=2, col=1)
fig.update_yaxes(title_text="Vibration (mm/s)", row=2, col=2)
fig.update_yaxes(title_text="Frequency", row=3, col=1)
fig.update_yaxes(title_text="Performance Score", row=3, col=2)

fig.show()

# Business insights and applications
print(f"\n ONE-CLASS SVM BUSINESS INSIGHTS:")

# Network security analysis
total_connections = len(network_df)
detected_intrusions = np.sum(network_df['ocsvm_prediction'])
actual_intrusions = np.sum(network_df['is_intrusion'])
detection_rate = np.sum((network_df['is_intrusion'] == 1) & (network_df['ocsvm_prediction'] == 1)) / actual_intrusions

print(f"\nNetwork Security Analysis:")
print(f"• Total connections analyzed: {total_connections:,}")
print(f"• Actual intrusions: {actual_intrusions}")
print(f"• Detected as anomalous: {detected_intrusions}")
print(f"• Detection rate: {detection_rate*100:.1f}%")
print(f"• False positive rate: {(detected_intrusions - actual_intrusions)/total_connections*100:.2f}%")

# Calculate security ROI
avg_intrusion_cost = 50_000 # Average cost per successful intrusion
prevented_intrusions = actual_intrusions * detection_rate
security_savings = prevented_intrusions * avg_intrusion_cost

print(f"• Prevented intrusion cost: ${security_savings:,.0f}")

# Equipment maintenance analysis
total_readings = len(equipment_df)
detected_faults = np.sum(equipment_df['ocsvm_prediction'])
actual_faults = np.sum(equipment_df['is_fault'])
fault_detection_rate = np.sum((equipment_df['is_fault'] == 1) & (equipment_df['ocsvm_prediction'] == 1)) / actual_faults

print(f"\nEquipment Maintenance Analysis:")
print(f"• Total sensor readings: {total_readings:,}")
print(f"• Actual faults: {actual_faults}")
print(f"• Detected as anomalous: {detected_faults}")
print(f"• Fault detection rate: {fault_detection_rate*100:.1f}%")

# Calculate maintenance ROI
avg_unplanned_downtime_cost = 100_000 # Cost per unplanned downtime event
avg_planned_maintenance_cost = 5_000 # Cost per planned maintenance
prevented_downtime = actual_faults * fault_detection_rate
maintenance_savings = prevented_downtime * (avg_unplanned_downtime_cost - avg_planned_maintenance_cost)

print(f"• Prevented downtime events: {prevented_downtime:.0f}")
print(f"• Maintenance cost savings: ${maintenance_savings:,.0f}")

# System implementation costs
ocsvm_implementation_cost = 200_000 # Initial setup
annual_operational_cost = 75_000 # Annual monitoring and maintenance

total_annual_benefits = security_savings + maintenance_savings
net_annual_benefits = total_annual_benefits - annual_operational_cost
roi = (net_annual_benefits - ocsvm_implementation_cost) / ocsvm_implementation_cost

print(f"\n ONE-CLASS SVM IMPLEMENTATION ROI:")
print(f"• Security savings: ${security_savings:,.0f}")
print(f"• Maintenance savings: ${maintenance_savings:,.0f}")
print(f"• Total annual benefits: ${total_annual_benefits:,.0f}")
print(f"• Implementation cost: ${ocsvm_implementation_cost:,.0f}")
print(f"• Annual operational cost: ${annual_operational_cost:,.0f}")
print(f"• Net annual benefits: ${net_annual_benefits:,.0f}")
print(f"• ROI: {roi*100:.0f}%")
print(f"• Payback period: {ocsvm_implementation_cost/net_annual_benefits*12:.1f} months")

# Kernel comparison insights
kernel_performance = results_df.groupby('kernel')['f1'].max().sort_values(ascending=False)
print(f"\nKernel Performance Ranking:")
for kernel, f1_score in kernel_performance.items():
 print(f"• {kernel}: {f1_score:.3f}")

print(f"\n Cross-Reference Learning Path:")
print(f"• Foundation: Tier5_SVM.ipynb (support vector machine concepts)")
print(f"• Comparison: Tier6_IsolationForest.ipynb (tree-based anomaly detection)")
print(f"• Comparison: Tier6_StatAnomaly.ipynb (statistical methods)")
print(f"• Advanced: Advanced_EnsembleAnomalyDetection.ipynb (combining methods)")
print(f"• Application: Advanced_RealtimeAnomalyDetection.ipynb (streaming data)")